## Bertopic model on Sentence-level, as 1st step for semantic role labeling (srl)

In [1]:
# install all packages
!pip install bertopic numpy matplotlib scikit-learn pandas datashader scikit-image requests # jinja2 (maybe need to reinstall.. Commented out because of RuntimeWarning (forking). # numba==0.56.4 need older version of numba for seeded lists in guided topic model
!pip install datamapplot
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install --upgrade cupy-cuda11x -f https://pip.cupy.dev/aarch64
from google.colab import drive
drive.mount('/content/drive')


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in links: https://pip.cupy.dev/aarch64
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load all packages
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import csv
from sentence_transformers import SentenceTransformer
from umap import UMAP
#import datamapplot
from hdbscan import HDBSCAN



ERROR:ptxcompiler.patch:Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba


In [3]:
# data preparation
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/coal_data_filtered_sentence_4.csv'
df = pd.read_csv(path_to_csv)
df_subset = df['sentence'].head(5000)
df_subset_time = df['Date'].head(5000)
df_subset_id = df['Unnamed: 0.1'].head(5000)


<ipython-input-3-bdb5650c9a86>:3: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_csv)


In [ ]:
print(df_subset)

0       AS WITH many a passion, one\'s earliest beginn...
1       When the door of my family home first opened a...
2       The world at my door was also that of the Tabl...
3       It was adjacent to its immense otherness - the...
4       In fact, you had only to cross the street, for...
                              ...                        
4995    One thing that South Africa can learn from Pak...
4996    A widening trust deficit between business and ...
4997    GLOBAL mining and metals group South32 this we...
4998    South32 said it would reduce its exposure to t...
4999    It said it planned to run SAEC as a stand-alon...
Name: sentence, Length: 5000, dtype: object


In [ ]:
# bertopic
#topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True) ## Might make sense to set calculate_probabilities=False if takes to much memory
#topics, probs = topic_model.fit_transform(df_subset)

#vectorizer_model = CountVectorizer(stop_words="english") # potentially add: ngram_range=(1, 3)
#topic_model.update_topics(df_subset, vectorizer_model=vectorizer_model)


# Map topics to article IDs
#topic_mapping = dict(zip(df_subset_id, topics))

# Print the first few article IDs and their corresponding topics
#for article_id, topic_id in topic_mapping.items():
#    print(f"Article ID: {article_id}, Topic ID: {topic_id}")

2024-06-09 17:13:07,367 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2024-06-09 17:14:59,166 - BERTopic - Embedding - Completed ✓
2024-06-09 17:14:59,168 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-09 17:15:44,147 - BERTopic - Dimensionality - Completed ✓
2024-06-09 17:15:44,149 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-09 17:15:46,358 - BERTopic - Cluster - Completed ✓
2024-06-09 17:15:46,377 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-09 17:15:46,640 - BERTopic - Representation - Completed ✓


Article ID: 2, Topic ID: 27
Article ID: 3, Topic ID: 0
Article ID: 4, Topic ID: 14
Article ID: 5, Topic ID: 18
Article ID: 6, Topic ID: 57
Article ID: 8, Topic ID: 7
Article ID: 9, Topic ID: -1
Article ID: 10, Topic ID: -1
Article ID: 11, Topic ID: -1
Article ID: 12, Topic ID: -1
Article ID: 13, Topic ID: 82
Article ID: 14, Topic ID: 82
Article ID: 15, Topic ID: -1
Article ID: 16, Topic ID: 44
Article ID: 17, Topic ID: 1
Article ID: 18, Topic ID: -1
Article ID: 19, Topic ID: 20
Article ID: 26, Topic ID: 28
Article ID: 27, Topic ID: -1
Article ID: 30, Topic ID: 82
Article ID: 31, Topic ID: 8
Article ID: 32, Topic ID: 5
Article ID: 33, Topic ID: -1
Article ID: 34, Topic ID: -1
Article ID: 35, Topic ID: 37
Article ID: 36, Topic ID: 19
Article ID: 37, Topic ID: -1
Article ID: 38, Topic ID: -1
Article ID: 39, Topic ID: -1
Article ID: 44, Topic ID: 6
Article ID: 46, Topic ID: 14
Article ID: 47, Topic ID: -1
Article ID: 48, Topic ID: -1
Article ID: 52, Topic ID: -1
Article ID: 55, Topic ID: -

In [13]:


# Assuming 'sentence' contains the sentences and 'article_id' contains the IDs for each article
df_subset = df[['Unnamed: 0.1', 'sentence']].head(50000)

# Instantiate BERTopic model
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

# Fit the model and transform the sentences into topics
topics, probs = topic_model.fit_transform(df_subset['sentence'])

#result_df = pd.DataFrame({'sentence': df_subset['sentence'], 'topic': topics, 'Unnamed: 0.1': df_subset['Unnamed: 0.1']})

# Create a mapping between sentences and article IDs
#sentence_article_mapping = dict(zip(df_subset['sentence'], df_subset['Unnamed: 0.1']))

# Map topics to article IDs
#topic_mapping = {}
#for sentence, topic_id in zip(df_subset['sentence'], topics):
#    article_id = sentence_article_mapping[sentence]
#    if article_id not in topic_mapping:
#        topic_mapping[article_id] = [topic_id]
#    else:
#        topic_mapping[article_id].append(topic_id)

# Print the first few article IDs and their corresponding topics
#for article_id, topic_ids in topic_mapping.items():
#    print(f"Article ID: {article_id}, Topic IDs: {topic_ids}")


2024-06-10 10:32:55,227 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2024-06-10 10:50:05,852 - BERTopic - Embedding - Completed ✓
2024-06-10 10:50:05,854 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-10 10:51:14,502 - BERTopic - Dimensionality - Completed ✓
2024-06-10 10:51:14,506 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
2024-06-10 11:15:03,677 - BERTopic - Cluster - Completed ✓
2024-06-10 11:15:03,715 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-10 11:15:05,623 - BERTopic - Representation - Completed ✓


In [14]:
vectorizer_model = CountVectorizer(stop_words="english") # potentially add: ngram_range=(1, 3)
topic_model.update_topics(df_subset['sentence'], vectorizer_model=vectorizer_model)

## Reduce outlier topics (the -1 topic) - https://maartengr.github.io/BERTopic/getting_started/outlier_reduction/outlier_reduction.html#embeddings
# Use the "c-TF-IDF" strategy with a threshold
new_topics = topic_model.reduce_outliers(df_subset['sentence'], topics , strategy="c-tf-idf", threshold=0.1)

# Reduce all outliers that are left with the "distributions" strategy
new_topics = topic_model.reduce_outliers(df_subset['sentence'], new_topics, strategy="distributions")

topic_model.update_topics(df_subset['sentence'], topics=new_topics)

result_df = pd.DataFrame({'sentence': df_subset['sentence'], 'topic': topics, 'Unnamed: 0.1': df_subset['Unnamed: 0.1']})

# Create a mapping between sentences and article IDs
sentence_article_mapping = dict(zip(df_subset['sentence'], df_subset['Unnamed: 0.1']))

# Map topics to article IDs
topic_mapping = {}
for sentence, topic_id in zip(df_subset['sentence'], topics):
    article_id = sentence_article_mapping[sentence]
    if article_id not in topic_mapping:
        topic_mapping[article_id] = [topic_id]
    else:
        topic_mapping[article_id].append(topic_id)

# Print the first few article IDs and their corresponding topics
for article_id, topic_ids in topic_mapping.items():
    print(f"Article ID: {article_id}, Topic IDs: {topic_ids}")


# Now for the Semantic role labeling need the following dataframe:
# Initialize an empty DataFrame
result_df = pd.DataFrame(columns=['article_id', 'sentence', 'topic'])

# Iterate over the topic mappings and populate the DataFrame
for article_id, topics in topic_mapping.items():
    for topic_id in topics:
        # Retrieve the sentence corresponding to the article ID
        sentence = list(sentence_article_mapping.keys())[list(sentence_article_mapping.values()).index(article_id)]
        # Append a row to the DataFrame
        result_df = result_df.append({'article_id': article_id, 'sentence': sentence, 'topic': topic_id}, ignore_index=True)

# Rename the 'Unnamed: 0.1' column to 'article_id'
result_df.rename(columns={'Unnamed: 0.1': 'article_id'}, inplace=True)


100%|██████████| 14/14 [00:03<00:00,  3.71it/s]
2024-06-10 11:15:54,314 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


Article ID: 2, Topic IDs: [-1, -1, 112, 112, 112, 112, 112, -1, -1, -1, -1, -1, -1, 112, -1, -1, -1, -1, -1, -1, -1, -1, 112, -1, -1, -1, -1, 112, 112, -1, -1, -1, 112, -1, -1, -1, -1, -1, -1, -1, -1, -1, 38, -1, -1, 112, 112, 112, 112, 112, -1, -1, 112, -1, -1, -1, -1, -1, -1, -1, -1, -1, 112, -1, -1, -1, 112, -1, 76, 112, -1, -1, -1, -1, 66, -1, -1, 112, 18, 59, 59, -1, -1, -1, -1, -1, -1, 0, 125]
Article ID: 2076, Topic IDs: [66, 66, 66, 66, 66, 393, 0, 32, -1, -1, -1, -1, -1, 66, -1, -1, 33, -1, -1, 102, -1, -1, -1, 35, 13, -1, -1, -1, 3, -1, -1, 7, -1, -1, 11, -1, 197, -1, -1]
Article ID: 3, Topic IDs: [12, 139, -1, -1, -1, -1, -1, -1, 4, 13, 4, -1, -1, 60, -1, -1, -1, 462, -1, -1, -1, 342, 14, -1, 9, 60, -1, 60]
Article ID: 4, Topic IDs: [181, -1, -1, 3, -1, 3, 159, 47, 3, -1, -1, -1, -1, -1, -1, -1, 172, -1, -1, -1, 34, 61, -1, -1, 34, -1, -1, 6, -1, -1, -1, -1, -1, -1, 3, -1, 34, 9, -1, -1, -1, 167, 9, 164, -1, -1, -1, 353, 61, -1, -1]
Article ID: 5, Topic IDs: [4, -1, 23, 23, 

In [15]:
# Save the model
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("content/drive/MyDrive/Colab Notebooks/topic_model_for_srl", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


In [21]:
print(result_df)


                                                sentence  topic  Unnamed: 0.1
0      AS WITH many a passion, one\'s earliest beginn...     -1             2
1      When the door of my family home first opened a...     -1             2
2      The world at my door was also that of the Tabl...    112             2
3      It was adjacent to its immense otherness - the...    112             2
4      In fact, you had only to cross the street, for...    112             2
...                                                  ...    ...           ...
49995  & I know that PV panels are expensive, but if ...     50          4426
49996  If there is an incentive, people will put up P...     50          4426
49997  Allowing SA to compete is the first priority T...      0          4427
49998  Growth is an insufficient but necessary condit...     12          4427
49999  To ensure that those investments result in gro...     -1          4427

[50000 rows x 3 columns]


In [24]:

result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/topic_model_srl.csv', index=False)


In [26]:
freq = topic_model.get_topic_info(); freq.head()
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,178,-1_lifile_ilizwe_befehl_do,"[lifile, ilizwe, befehl, do, why, humiliating,...",[When asked whether smaller increases for exec...
1,0,1015,0_south_africa_africans_african,"[south, africa, africans, african, africas, co...","[""Not only for me, but for all business in Sou..."
2,1,921,1_mining_mines_department_communities,"[mining, mines, department, communities, right...",[This is the untold story of many of the mines...
3,2,561,2_coal_quality_wet_export,"[coal, quality, wet, export, conveyor, burn, w...","[It\'s coal or nuclear., Coal is still very im..."
4,3,447,3_zuma_zumas_jacob_president,"[zuma, zumas, jacob, president, anc, president...",[Some money even found its way to a foundation...
...,...,...,...,...,...
483,482,27,482_reckons_litres_wuc_potable,"[reckons, litres, wuc, potable, amd, wucs, wat...",[WUC reckons it will produce 67 million litres...
484,483,18,483_ppp_ppps_partnerships_airports,"[ppp, ppps, partnerships, airports, chennai, e...","[Also, the government has decided to develop 2..."
485,484,75,484_ownership_jurisdictions_unable_cleanup,"[ownership, jurisdictions, unable, cleanup, st...",[Or is it a case of private ownership and a go...
486,485,50,485_studies_harries_uct_history,"[studies, harries, uct, history, university, h...",[In the words of then professor of African his...


In [ ]:
## NEXT APPROACH

In [42]:
#from transformers.pipelines import pipeline

In [4]:
df_subset = df[['Unnamed: 0.1', 'sentence']].head(100000)
docs = df_subset['sentence']
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words="english")
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)


# We reduce our embeddings to 2D as it will allows us to quickly iterate later on
reduced_embeddings = UMAP(n_neighbors=10, n_components=2,
                          min_dist=0.0, metric='cosine').fit_transform(embeddings)
hdbscan_model = HDBSCAN(min_cluster_size=200, min_samples = 2, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

# Train our topic model
topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model,
                       vectorizer_model=vectorizer_model, hdbscan_model=hdbscan_model, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(docs, embeddings)

representation_model = MaximalMarginalRelevance(diversity=0.2)
topic_model.update_topics(docs, representation_model=representation_model)

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("content/drive/MyDrive/Colab Notebooks/topic_model_for_srl2", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)



### create table

result_df = pd.DataFrame({'sentence': df_subset['sentence'], 'topic': topics, 'article_id': df_subset['Unnamed: 0.1']})
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/topic_model_srl2.csv', index=False)


Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Article ID: 2, Topic IDs: [14, -1, 51, -1, 51, 51, 51, -1, 19, -1, -1, -1, 42, 51, -1, -1, -1, 19, -1, 19, -1, 14, 51, 11, 51, 11, -1, 51, 51, 51, -1, -1, 51, -1, -1, -1, -1, -1, -1, -1, 11, -1, 27, -1, -1, 51, 51, 51, 51, -1, 51, -1, 51, -1, -1, 14, -1, 14, -1, -1, -1, -1, 51, -1, -1, 14, 51, -1, 14, 51, 11, 51, 51, 14, -1, -1, 51, 56, -1, -1, -1, -1, -1, -1, 51, -1, 8, 33]
Article ID: 11843, Topic IDs: [-1, 14, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 8, 0, 0, 10, 3, 10, -1, 0, 14, 0, 0, -1, 0, 0, 8, 4, -1, 0, 0, -1, 14, -1, 25, 25, -1, -1, 0, 10, 31, -1, -1, -1, 0, 0, 0, 50, -1, 39, -1, -1, -1, -1, -1, 14, -1, -1, 0, -1, 24, 25, -1, 8, -1, 10, 4, 4, 4, 0, 69, 31, 4, -1, -1, 10, -1, 25, 0, -1, -1, -1, -1, 14, -1, -1, -1, -1, 8, -1, 23, 13, -1, -1, 25, -1, -1, -1, -1, -1, 4, -1, 21, -1, 0, -1, -1, -1, 4, -1, 2, -1, 0, 8, -1, 31, -1, -1, -1, 25, -1, -1, -1, -1, -1, -1, -1, 14, -1, -1, -1, 8, -1, -1, 4, 8, -1, 25, 0, 4, -1, 25, -1, -1, -1, -1, -1, -1]
Article ID: 3, Topic IDs: [-1

AttributeError: 'DataFrame' object has no attribute 'append'

In [7]:
print(result_df)

                                                sentence  topic  article_id
0      AS WITH many a passion, one\'s earliest beginn...     14           2
1      When the door of my family home first opened a...     -1           2
2      The world at my door was also that of the Tabl...     51           2
3      It was adjacent to its immense otherness - the...     -1           2
4      In fact, you had only to cross the street, for...     51           2
...                                                  ...    ...         ...
99995  Mozambique has blocked a plan put forward by R...     -1       11921
99996  - Bloomberg Construction of the largest renewa...      6       11921
99997  The Suzlon Group has been given the go-ahead f...      6       11921
99998  The project reached financial closure in Decem...     62       11921
99999  - Londiwe Buthelezi The SA Democratic Teachers...     42       11921

[100000 rows x 3 columns]


In [10]:
# Get detailed topic information
topic_info = topic_model.get_topic_info()

# Print the topic information
print("Topic Information:")
print(topic_info)

# Display details of each topic
for topic_id in topic_info['Topic']:
    if topic_id == -1:
        continue  # Skip the outlier topic
    print(f"\nTopic {topic_id}:")
    print(topic_model.get_topic(topic_id))

topic_info.to_csv('/content/drive/MyDrive/Colab Notebooks/topic_info2.csv', index=False)

Topic Information:
    Topic  Count                                        Name  \
0      -1  46695                              -1_of_to_in_on   
1       0   3483        0_mining_resources_industry_minerals   
2       1   2927               1_rail_port_freight_transport   
3       2   2280                 2_share_earnings_jse_shares   
4       3   1941                   3_coal_export_rbct_prices   
..    ...    ...                                         ...   
67     66    260            66_bhp_billiton_assets_aluminium   
68     67    257    67_shedding_eskom_blackouts_loadshedding   
69     68    222           68_blackouts_outages_blackout_ups   
70     69    208                  69_rating_moody_fitch_debt   
71     70    200  70_regulatory_regulations_consultation_law   

                                       Representation  \
0     [of, to, in, on, said, be, at, we, coal, eskom]   
1   [mining, resources, industry, minerals, coal, ...   
2   [rail, port, freight, transport, loco

In [ ]:
representation_model = MaximalMarginalRelevance(diversity=0.2)
topic_model.update_topics(docs, representation_model=representation_model)


In [5]:
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings,
                                hide_document_hover=True, hide_annotations=True)


Output hidden; open in https://colab.research.google.com to view.

In [37]:
#new_topics = topic_model.reduce_outliers(docs, topics , strategy="c-tf-idf", threshold=0.1)
#topic_model.update_topics(docs, topics=new_topics)
#

2024-06-10 12:36:47,239 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [46]:
#representation_model = MaximalMarginalRelevance(diversity=0.2)
#topic_model.update_topics(docs, representation_model=representation_model)


In [47]:
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings,
                                hide_document_hover=True, hide_annotations=True)

Check this article out!
https://towardsdatascience.com/topics-per-class-using-bertopic-252314f2640
